# Lecture 11: Data analysis with ```pandas```

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In the Lecture 5, you have already looked into ```csv.reader``` which converts data from CSV files into python strings. Working with this type of data is not always convenient. 

### Pandas
 - library for data manipulation and analysis
 - imports data from CSV, JSON, SQL, Excel etc.
 - built on top of ```numpy```
 - most machine learning tools support ```pandas``` $\rightarrow$ used for data preprocessing in machine learning
 

## Reading the data file

### Isotope database

As an example, we will work with ```isotope_data.csv``` that was generated from https://www-nds.iaea.org/relnsd/vcharthtml/VChartHTML.html

This database contains information on physical properties of most of the known isotopes (but not all!)

In [ ]:
df = pd.read_csv("isotope_data.csv")

If you already know the content of the file and you don't need all the columns, you can read the file more efficiently with 

```usecols = ['column1', 'column2', 'column3']```

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df['Z']

In [ ]:
type(df['Z'])

### What is inside?

In [ ]:
print(df.info())

In [ ]:
df.describe()

In [ ]:
df.head()

We can also specify how many rows we want to see with head() (and tail()).

In [ ]:
df.head(3)

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

## Selecting data from the table

### Selecting columns

In [ ]:
df[["Z","N"]]

In [ ]:
df.iloc[:, 1]

In [ ]:
df.loc[:, "N"]

### Selecting rows

In [ ]:
df.loc[2:3]

In [ ]:
df.iloc[2:3]

### Selecting a range from the table

In [ ]:
df.loc[2:5, ["Z","N","symbol" ,"radius [fm]","half_life"]]

In [ ]:
df.iloc[2:6, [0,1,2,3,12]]

### Selecting by condition

In [ ]:
df[df['half_life']=='STABLE'] #selecting only stable isotopes

In [ ]:
df[df['symbol']=='Ra'][['symbol', 'radius [fm]',' half_life [s]']]

In [ ]:
df[(df['half_life']=='STABLE')&(df['N']>120)]

## Manipulating the dataframe 

### Making a reduced version of the original dataframe

In [ ]:
df.columns

In [ ]:
df_reduced = df[['Z', 'N', 'symbol', 'radius [fm]',' abundance [mole fract.]',' half_life [s]','atomic mass [AMU * 1.E-3 ]',' discovery']]


# please note, that doing df = df[["column1", "column2", ...]] is a bad style that leads to many possible errors!

In [ ]:
df_reduced

### Rename columns

In [ ]:
df_reduced.rename({'radius [fm]': 'R_fm', ' abundance [mole fract.]': 'abund.',' half_life [s]':'tau', 'atomic mass [AMU * 1.E-3 ]':'A', ' discovery':'year'}, axis=1)

In [ ]:
df_reduced = df_reduced.rename({'radius [fm]': 'R_fm', ' abundance [mole fract.]': 'abund.',' half_life [s]':'tau', 'atomic mass [AMU * 1.E-3 ]':'A', ' discovery':'year'}, axis=1)

### Perform an operation on column contents

In [ ]:
df_reduced['A'] = df_reduced['A']/1e6

In [ ]:
df_reduced

You can also add/subtract/multiply/divide columns together (in this case it doesn't make sense, but as an example)

In [ ]:
df_reduced['R_fm'] * df_reduced['tau']

### Add a row 

New rows can be added with concat().

In [ ]:
df_reduced.columns

In [ ]:
new_isotope = pd.DataFrame({"Z": [0], "N": [1], "symbol": ['n'], "R_fm": [-0.1149], "abund.": ['NaN'], "tau": [6.139000e+02], "A": [1.008665], "year": ['NaN']})
new_isotope

In [ ]:
df_newrow = pd.concat([df_reduced, new_isotope], ignore_index=True)
df_newrow

### Add a column, modify contents of a column

In [ ]:
df_reduced

In [ ]:
df_reduced['stability']=df_reduced['tau'].isnull()

In [ ]:
df_reduced

In [ ]:
df_reduced['stability2']=df_reduced['tau'].isnull()

In [ ]:
df_reduced

In [ ]:
df_reduced.loc[df_reduced.shape[0]] = np.ones(df_reduced.shape[1])

In [ ]:
df_reduced

### Remove a column

In [ ]:
df_reduced=df_reduced.drop('stability2',axis=1)

In [ ]:
df_reduced

### Remove a row

In [ ]:
df_reduced=df_reduced.drop([df_reduced.shape[0]-1])

In [ ]:
df_reduced

### Column statistics

Note that instances of `NaN` will be automatically removed from the calculation.

In [ ]:
df_reduced['R_fm'].mean()

We can specify that we want to keep `NaN` entries.

In [ ]:
df_reduced['R_fm'].mean(skipna=False)

If we want the mean of a row, rather than a column, we can do df.mean(axis=1).

### Saving into a file

In [ ]:
df_stable = df_reduced[df_reduced['stability']==1]
df_stable.to_csv("stable_isotopes.csv")


df_unstable = df_reduced[df_reduced['stability']==0]
df_unstable.to_csv("unstable_isotopes.csv")

## Quick analysis tools

### Histogram

In [ ]:
df_stable["year"].hist()

In [ ]:
df_unstable["year"].hist()

In [ ]:
# same plot with matplotlib

plt.hist(df_stable["year"], bins=20)
plt.xlabel("Year")
plt.ylabel ("Number of discovered stable isotopes")

### Scatter plot

In [ ]:
df_stable[["R_fm", "A"]].plot.scatter("A", "R_fm")

In [ ]:
df_unstable[["R_fm", "A"]].plot.scatter("A", "R_fm")
#plt.plot(df_unstable["A"],1.2*df_unstable['A']**(1/3))
#plt.xlim([0,50])

### Bar plot

In [ ]:
df_stable[df_stable['symbol']=='Fe']

df_stable[df_stable['symbol']=='Fe'].plot.bar("N","abund.")

### Example 1: finding the most long-living Radium isotope

In [ ]:
df_unstable.describe()

In [ ]:
df_radium = df_unstable[df_unstable['symbol']=='Ra']

In [ ]:
df_radium.loc[df_radium['tau'].idxmax()]

### Example 2: Finding only unstable elements

In [ ]:
df_unstable[~df_unstable['symbol'].isin(df_stable['symbol'])]

### Example 3: ten largest nuclei 

In [ ]:
df_unstable.sort_values(by='R_fm',na_position='first')

In [ ]:
df_unstable.sort_values(by='R_fm',na_position='first').iloc[-10:]

### Example 4: Dealing with missing data

In [ ]:
df_unstable.dropna()

In [ ]:
df_unstable.isna()

# Additional resources

It is worth having a look at "10 minutes to pandas" in the documentation: https://pandas.pydata.org/docs/user_guide/10min.html